## INTRODUCTION TO V-REP

V-REP is a powerful 3D robot simulator, which features several versatile calculation modules (inverse kinematics, Physics/dynamics, collision detections, minimum distance calculations, path planning, etc.), a distributed control architecture (unlimited number of control scripts, threaded or non threaded), and several extension mechanisms (plug-ins, custom client application, etc.).It offers a multitude of functionalities that can be easily integrated and combined through an exhaustive API and script functionality.

V-REP is the tool for fast prototyping and verification, remote monitoring, fast algorithm development, robotics related education, and simulation of factory automation systems. A V-REP plug-in was specifically developed to allow an intuitive, easy and realistic Khepera robot simulation.

On V-REP's homepage the program is described as

    "the Swiss army knife among robot simulators: you won't find a simulator with more functions, features, or more elaborate APIs."


V-REP's strength comes from several features:

1. V-REP provides a unified framework combining many powerful internal and external libraries that are often useful for robotics simulations.
2. V-REP is highly extensible.
3. V-REP is cross-platform, mostly open-source, and provides a free educational license.

### INTERACTION WITH V-REP

#### V-REP GUI INTERFACE

<img src="userInterface.jpg">


There are two ways to interact with the V-REP simulator, they are through:

1. REGULAR V-REP API: These are all functions that can either be called directly from a custom C/C++ plugin or through a Lua embedded script.  V-REP functions and constants can easily be recognized from their "sim"- or "_sim"-prefix (e.g. sim.handleCollision)
http://www.coppeliarobotics.com/helpFiles/en/apiOverview.htm

2. REMOTE V-REP API: this  is how V-REP enables scripts and programs written in other languages (MATLAB, Java, Python, etc.) to interact with a V-REP simulation from an external application or a remote hardware (e.g. real robot, remote computer, etc.). The remote API functions are interacting with V-REP via socket communication (or, optionally, via shared memory) in a way that reduces lag and network load to a great extent. The remote API can let one or several external applications interact with V-REP in a synchronous* or asynchronous* way (asynchronous by default), and even remote control of the simulator is supported (e.g. remotely loading a scene, starting, pausing or stopping a simulation for instance).

#### REMOTE API

The remote API functionality comes in 2 separate entities, that are interacting via socket communication:

- the client side (i.e. your application): the remote API on the client side is available for many different programming languages. Currently following languages are supported: C/C++, Python, Java, Matlab, Octave and Lua. 

- the server side (i.e. V-REP): the remote API on the server side is implemented via a V-REP plugin that should be loaded by V-REP by default: v_repExtRemoteApi.dll, libv_repExtRemoteApi.dylib or libv_repExtRemoteApi.so. The plugin project files are located here. Instructions on how to enabled the remote API on the server side are given here.

#### STARTING THE REMOTE API

There are two ways to start the remote API:

- At start up the simulator looks for a file named **remoteApiConnections.txt**, based on the parameters set in the file an appropriate connection will be set.

- From within a script. A temporary remote API server service can be started or stopped with following 2 custom Lua functions (the 2 functions are exported by the plugin): 
    * simRemoteApi.start(port_number)
    * simRemoteApi.stop(port_number)
    
#### LOOKING AT A SCRIPT TO INTERACT WITH V-REP REMOTELY

**simpleTest.py**

#### LOOKING AT THE PIONEER 3DX

<img src="robot-mobile-pioneer-3-dx.jpg">

Pioneer 3 DX is a compact differential-drive mobile robot. It arrives fully assembled with embedded controller, motors with 500-tick encoders, 19cm wheels, tough aluminum body, 8 forward-facing ultrasonic (sonar) sensors, 8 optional rear-facing sonar, 1, 2 or 3 hot-swappable batteries, and our complete software development kit. Add an optional internal computer or your own laptop and the robot is ready to go.  

Additional sensors, manipulators and other accessories are available to customize your Pioneer for your specific tasks.

<img src="pioneer_manipulator.jpeg">

#### PIONEER IN V-REP

#### DIFFERENTIAL WHEEL DRIVE

A differential wheeled robot is a mobile robot whose movement is based on two separately driven wheels placed on either side of the robot body. It can thus change its direction by varying the relative rate of rotation of its wheels and hence does not require an additional steering motion.

###### FIND: the mathematical derivation for the differential drive

#### LOOKING AT A SCRIPT TO INTERACT WITH V-REP REMOTELY AND CONTROL THE ROBOT

**testDrive.py**


## NEXT SESSION

* Add at least two sensors to the robot and print out its data.
* Braitenberg algorithm for obstacle aviodance


### CREDITS

- https://www.k-team.com/mobile-robotics-products/v-rep
- http://hades.mech.northwestern.edu/index.php/V-REP_Introduction
- http://www.coppeliarobotics.com/helpFiles/en/remoteApiOverview.htm
- http://www.coppeliarobotics.com/helpFiles/en/apiOverview.htm
- http://www.coppeliarobotics.com/helpFiles/en/remoteApiServerSide.htm
- http://www.coppeliarobotics.com/helpFiles/en/remoteApiModusOperandi.htm
- http://www.mobilerobots.com/ResearchRobots/PioneerP3DX.aspx
- https://en.wikipedia.org/wiki/Differential_wheeled_robot

In [ ]:
# Copyright 2006-2017 Coppelia Robotics GmbH. All rights reserved. 
# marc@coppeliarobotics.com
# www.coppeliarobotics.com
# 
# -------------------------------------------------------------------
# THIS FILE IS DISTRIBUTED "AS IS", WITHOUT ANY EXPRESS OR IMPLIED
# WARRANTY. THE USER WILL USE IT AT HIS/HER OWN RISK. THE ORIGINAL
# AUTHORS AND COPPELIA ROBOTICS GMBH WILL NOT BE LIABLE FOR DATA LOSS,
# DAMAGES, LOSS OF PROFITS OR ANY OTHER KIND OF LOSS WHILE USING OR
# MISUSING THIS SOFTWARE.
# 
# You are free to use/modify/distribute this file for whatever purpose!
# -------------------------------------------------------------------
#
# This file was automatically created for V-REP release V3.4.0 rev. 1 on April 5th 2017

# Make sure to have the server side running in V-REP: 
# in a child script of a V-REP scene, add following command
# to be executed just once, at simulation start:
#
# simExtRemoteApiStart(19999)
#
# then start simulation, and run this program.
#
# IMPORTANT: for each successful call to simxStart, there
# should be a corresponding call to simxFinish at the end!

try:
    import vrep
except:
    print ('--------------------------------------------------------------')
    print ('"vrep.py" could not be imported. This means very probably that')
    print ('either "vrep.py" or the remoteApi library could not be found.')
    print ('Make sure both are in the same folder as this file,')
    print ('or appropriately adjust the file "vrep.py"')
    print ('--------------------------------------------------------------')
    print ('')

import time

print ('Program started')
vrep.simxFinish(-1) # just in case, close all opened connections
clientID=vrep.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to V-REP
if clientID!=-1:
    print ('Connected to remote API server')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res,objs=vrep.simxGetObjects(clientID,vrep.sim_handle_all,vrep.simx_opmode_blocking)
    if res==vrep.simx_return_ok:
        print ('Number of objects in the scene: ',len(objs))
    else:
        print ('Remote API function call returned with error code: ',res)

    time.sleep(2)

    # Now retrieve streaming data (i.e. in a non-blocking fashion):
    startTime=time.time()
    vrep.simxGetIntegerParameter(clientID,vrep.sim_intparam_mouse_x,vrep.simx_opmode_streaming) # Initialize streaming
    while time.time()-startTime < 5:
        returnCode,data=vrep.simxGetIntegerParameter(clientID,vrep.sim_intparam_mouse_x,vrep.simx_opmode_buffer) # Try to retrieve the streamed data
        if returnCode==vrep.simx_return_ok: # After initialization of streaming, it will take a few ms before the first value arrives, so check the return code
            print ('Mouse position x: ',data) # Mouse position x is actualized when the cursor is over V-REP's window
        time.sleep(0.005)

    # Now send some data to V-REP in a non-blocking fashion:
    vrep.simxAddStatusbarMessage(clientID,'Hello V-REP!',vrep.simx_opmode_oneshot)

    # Before closing the connection to V-REP, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
    vrep.simxGetPingTime(clientID)

    # Now close the connection to V-REP:
    vrep.simxFinish(clientID)
else:
    print ('Failed connecting to remote API server')
print ('Program ended')



In [ ]:
import vrep                  #V-rep library
import sys
import time                #used to keep track of time
import numpy as np         #array library
import math
import matplotlib as mpl  

vrep.simxFinish(-1)

clientID = vrep.simxStart('127.0.0.1', 19999, True, True, 5000, 5)

if clientID!= -1:
    print("Connected to remote server")
else:
    print('Connection not successful')
    sys.exit('Could not connect')

errorCode,left_motor_handle=vrep.simxGetObjectHandle(clientID,'Pioneer_p3dx_leftMotor',vrep.simx_opmode_oneshot_wait)

errorCode,right_motor_handle=vrep.simxGetObjectHandle(clientID,'Pioneer_p3dx_rightMotor',vrep.simx_opmode_oneshot_wait)

if errorCode == -1:
    print('Can not find left or right motor')
    sys.exit()

errorCode=vrep.simxSetJointTargetVelocity(clientID,left_motor_handle,0.2, vrep.simx_opmode_oneshot_wait)
errorCode=vrep.simxSetJointTargetVelocity(clientID,right_motor_handle,0.2, vrep.simx_opmode_oneshot_wait)


sensor_h=[] #empty list for handles
sensor_val=np.array([]) #empty array for sensor measurements

PI = math.pi
#orientation of all the sensors: 
sensor_loc=np.array([-PI/2, -50/180.0*PI,-30/180.0*PI,-10/180.0*PI,10/180.0*PI,30/180.0*PI,50/180.0*PI,PI/2,PI/2,130/180.0*PI,150/180.0*PI,170/180.0*PI,-170/180.0*PI,-150/180.0*PI,-130/180.0*PI,-PI/2]) 

#for loop to retrieve sensor arrays and initiate sensors
for x in range(1,16+1):
        errorCode,sensor_handle=vrep.simxGetObjectHandle(clientID,'Pioneer_p3dx_ultrasonicSensor'+str(x),vrep.simx_opmode_oneshot_wait)
        sensor_h.append(sensor_handle) #keep list of handles        
        errorCode,detectionState,detectedPoint,detectedObjectHandle,detectedSurfaceNormalVector=vrep.simxReadProximitySensor(clientID,sensor_handle,vrep.simx_opmode_streaming)                
        sensor_val=np.append(sensor_val,np.linalg.norm(detectedPoint)) #get list of values

print(sensor_val)

time.sleep(10)
errorCode=vrep.simxSetJointTargetVelocity(clientID,left_motor_handle,0, vrep.simx_opmode_oneshot_wait)
errorCode=vrep.simxSetJointTargetVelocity(clientID,right_motor_handle,0, vrep.simx_opmode_oneshot_wait)

